# Quality Assessment
Now it's your turn. Follow the steps on the platform and use what you've learnt to see how reliable the data is.

In [2]:
import pandas as pd

Load our cleaned DataFrames

In [3]:
# orders_cl.csv
url = "https://drive.google.com/file/d/1Tla62vfu__kCqvgypZyVt2S9VuC016yH/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders_cl = pd.read_csv(path)

# orderlines_cl.csv
url = "https://drive.google.com/file/d/1OhtkQS2fwOYdzfd-qPh7im35iLc-L9TA/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines_cl = pd.read_csv(path)

# products_cl.csv
url = "https://drive.google.com/file/d/1s7Lai4NSlsYjGEPg1QSOUJobNYVsZBOJ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_csv(path)

In [4]:
orders_qu = orders_cl.copy()

In [5]:
orderlines_qu = orderlines_cl.copy()

In [6]:
products_qu = products_cl.copy()

## 1.&nbsp; Define Pandas display format

In [7]:
# your code here
pd.set_option("display.float_format", lambda x: "%.2f" % x)
pd.set_option("display.max_rows", 1000)

## 2.&nbsp; Exclude unwanted orders

In [8]:
# your code here
orders_qu["state"].value_counts()

state
Shopping Basket    117809
Completed           46605
Place Order         40883
Pending             14374
Cancelled            7233
Name: count, dtype: int64

Using only completed orders:

In [9]:
# using just completed
completed_state_mask = orders_qu["state"] == "Completed"

In [10]:
# using more state
state_mask = orders_qu["state"].isin(["Completed", "Place Order", "Pending"])

Identifying the orders according to the state:

In [11]:
# in case you want to keep id_s
dropped_ids = orders_qu.loc[~completed_state_mask, "order_id"]

In [12]:
# orders_qu["oder_id"].isin()

In [13]:
completed_orders_df = orders_qu.loc[completed_state_mask, :]

In [14]:
completed_orders_df

,order_id,created_date,total_paid,state
1,241423,2017-11-06 13:10:02,136.15,Completed
2,242832,2017-12-31 17:40:03,15.76,Completed
3,243330,2017-02-16 10:59:38,84.98,Completed
5,245275,2017-06-28 11:35:37,149.00,Completed
6,245595,2017-01-21 12:52:47,112.97,Completed
...,...,...,...,...
226544,527042,2018-03-14 11:47:50,18.98,Completed
226572,527070,2018-03-14 11:50:48,24.97,Completed
226576,527074,2018-03-14 11:51:42,24.97,Completed
226598,527096,2018-03-14 11:58:40,34.96,Completed


In [15]:
completed_orders_df["order_id"].nunique()

46605

In [16]:
# orders_cl["order_id"].nunique()

In [17]:
orderlines_qu["id_order"].nunique()

170213

Using inner merge:

In [18]:
orders_orderlines = completed_orders_df.merge(orderlines_qu, how="inner", left_on="order_id", right_on="id_order")#[["order_id"]]

In [19]:
# reference list of shared / common order_ids
order_ids = orders_orderlines["order_id"].unique()

In [20]:
orders_qu = orders_qu.loc[orders_qu["order_id"].isin(order_ids)]

In [21]:
orderlines_qu = orderlines_qu.loc[orderlines_qu["id_order"].isin(order_ids)]

Using set methods:

In [22]:
# orders_set = set(orders_qu.loc[completed_state_mask, "order_id"])

In [23]:
orders_set = set(completed_orders_df["order_id"])

In [24]:
orderlines_set = set(orderlines_qu["id_order"].unique())

In [25]:
common_order_ids = orders_set.intersection(orderlines_set)

In [26]:
orders_qu = orders_qu.loc[orders_qu["order_id"].isin(common_order_ids)]

In [27]:
orderlines_qu = orderlines_qu.loc[orderlines_qu["id_order"].isin(common_order_ids), :]

---

In [28]:
orders_qu["order_id"].nunique(), orderlines_qu["id_order"].nunique()

(43064, 43064)

## 3.&nbsp; Exclude orders with unknown products


In [29]:
# your code here
orderlines_qu["sku"].nunique(), products_qu["sku"].nunique()

(5400, 9992)

In [30]:
orderlines_qu["sku"].sort_values().unique()

array(['8MO0001-A', '8MO0003-A', '8MO0007', ..., 'ZEP0007', 'ZEP0008',
       'par0072'], shape=(5400,), dtype=object)

In [31]:
products_qu["sku"].duplicated().sum()

np.int64(0)

In [32]:
orderlines_products = orderlines_qu.merge(products_qu, how="left", on="sku")#[["id_order", "sku", "name"]]

In [33]:
orderlines_products.sample(3)

,id,id_order,product_id,product_quantity,sku,unit_price,date,name,desc,price,in_stock,type
38928,1477454,455534,0,1,ALL0010,16.99,2017-12-05 09:38:40,PowerCube Original USB Regleta Allocacoc wall ...,Wall socket cube-shaped with 4 to 250V electri...,18.95,1.00,5395
40255,1495012,462307,0,1,WDT0140,84.99,2017-12-13 22:06:44,"Red 2TB WD 35 ""Mac PC hard drive and NAS",Western Digital hard drive designed for NAS 2T...,99.00,1.00,12655397
34802,1439600,440262,0,3,CRU0021,79.99,2017-11-24 17:12:34,Crucial Mac Memory 8GB 1600MHZ DDR3 SO-DIMM,8GB RAM 135V / 15V MacBook iMac (2012/2013) Ma...,79.99,1.00,1364


In [34]:
# example
orderlines_products.loc[orderlines_products["id_order"] == 416674]

,id,id_order,product_id,product_quantity,sku,unit_price,date,name,desc,price,in_stock,type
28289,1386543,416674,0,1,TRA0011,45.28,2017-10-24 12:14:38,NaN,NaN,NaN,NaN,NaN


In [35]:
orderlines_products["name"].isna().value_counts()

name
False    55320
True      1394
Name: count, dtype: int64

In [36]:
# step in between: identifying the SKUs that are unknown and put them in a list
# goal is to create a reference list of affected id_orders to clean orders and orderlines from orders with unknown products

# we can also skip this and grab the unique id_orders right away - just to illustrate and in case you want to double check in products
unknown_products_sku = orderlines_products.loc[orderlines_products["name"].isna(), "sku"]

In [37]:
products_qu.loc[products_qu["sku"].isin(unknown_products_sku)]

,sku,name,desc,price,in_stock,type


In [38]:
# refernce list of orders to delete because they contain an unknown sku
orders_to_delete = orderlines_products.loc[orderlines_products["sku"].isin(unknown_products_sku), "id_order"].unique()

In [39]:
orderlines_products.loc[orderlines_products['price'].isna(), "id_order"].unique()

array([299638, 299706, 299712, ..., 526106, 526122, 526363], shape=(1363,))

In [40]:
# refernce list of orders to delete because they contain an unknown sku, identified by the NaN value e.g. in name
orders_to_delete = orderlines_products.loc[orderlines_products["name"].isna(), "id_order"].unique()

In [41]:
orders_qu = orders_qu.loc[~orders_qu["order_id"].isin(orders_to_delete)]

In [42]:
orderlines_qu = orderlines_qu.loc[~orderlines_qu["id_order"].isin(orders_to_delete)]

In [43]:
orders_qu["order_id"].sort_values().nunique() == orderlines_qu["id_order"].sort_values().nunique()

True

Exploring why we need to get rid of the whole order: difference in sum of unit_prices and total_paid per order

In [44]:
orderlines_test = orderlines_qu.copy()

In [45]:
orderlines_test["unit_price_total"] = (orderlines_test["product_quantity"]) * (orderlines_test["unit_price"])

In [46]:
orderlines_test.groupby("id_order")["unit_price_total"].sum()

id_order
241423   129.16
242832    10.77
243330    77.99
245275   149.00
245595   105.98
          ...  
527042    13.99
527070    19.98
527074    19.98
527096    29.97
527112     9.99
Name: unit_price_total, Length: 41701, dtype: float64

In [47]:
# orders_qu.merge(orderlines_test, how="inner", left_on="order_id", right_on="id_order")[["order_id", "total_paid", "unit_price_total"]]

---

**Cleaning sample steps:**
1.  orders:
- no duplicates
- **dropped rows** with NaN in total_paid * (result: less order_ids than source file)
- converted created_date to datetime
- (total_paid already float)
2. orderlines:  
- no duplicates
- no missing values in orderlines
- converted to datetime
- **dropped entire id_orders with at least one corrupted unit_price** *, then converted to numeric (result: less id_orders)
3. products:
- dropped duplicates
- missing values in desc - overwritten with name *
- missing values in type - left alone / overwrite with empty string *
- **dropped missing values in price** - very valuable column, needs to be rock solid info (result: less SKUs)
- dropped promo price due to too many corrupted values

(**Bold** = steps that affect other tables, "infectious disease";
Asterisk * = steps you may have other approaches for)

---
**Quality assessment:**
1. Exclude unwanted orders: decision to only work with "Completed orders" *
- **Step 1:** Identify the **completed orders by their order_ids**.

(Use a mask in .loc and grab the order_id column.)
- **Step 2:** Use them to filter the matching records in both dataframes, orders_qu and orderlines_qu.

(Use either an inner merge or set method intersection() to find the intersected / common order_ids as a reference list).
- **Step 3:** Overwrite the original orders_qu and orderlines_qu to keep only the order_ids of completed orders for the next steps.

(Use a .loc and .isin with the reference list)

2. Exclude orders with unknown products
- **Step 1:** Identify the **corrupted order lines**.

(Use a left merge making orderlines the dominant tables - the NaNs in columns of products reveal the affected SKUs. Note: An inner merge would this time get rid of only the single affected rows, not the whole order because you merge on SKU.)
- **Step 2:** Identify the **orders** those lines belong to — those orders are now also corrupted.

(Use a mask with .isna() in any products_cl related column like "name" and store the SKUs in a varibale. Use that variable to identify all affected/corrupted unique id_orders.)
- **Step 3:** Identify **all the other order lines** within those corrupted orders.

(Use affected unique id_orders to update the orderlines_qu and orders_qu dataframes so that they in the end only contain id_orders / order_ids that are not in that list of corrupted order_ids.)

---

In [48]:
orders_qu["order_id"].nunique(), orderlines_qu["id_order"].nunique()

(41701, 41701)

## 4.&nbsp; Explore the revenue from different tables

#### Step 1:
Create the `unit_price_total` as `orderlines.unit_price` * `orderlines.product_quantity`

In [49]:
# your code here

#### Step 2:
Group by `id_order`, summarising by the sum of `unit_price_total`

In [50]:
# your code here

### What is the average difference between `total_paid` and `unit_price_total`?

In [51]:
# your code here

### What is the distribution of these differences?

In [52]:
# your code here

### Can all the differences be explained by shipping costs? If not, what are other plausible explanations?

In [53]:
# your explanation here

### If there are differences that you can’t explain: what should you do with these orders?

In [54]:
# your code here

## 5.&nbsp; Become confident about your dataset

Let's hear your thoughts about what you discovered.

In [55]:
# your thoughts & code here

Do not forget to download your quality controlled DataFrames

In [56]:
#from google.colab import files

#orders_quality_controlled.to_csv("orders_qu.csv", index=False)
#files.download("orders_qu.csv")

#orderlines_quality_controlled.to_csv("orderlines_qu.csv", index=False)
#files.download("orderlines_qu.csv")

In [ ]:
import re

def categorize_product(row):
    """
    Categorize products based on name and description.
    Products with the same 'type' value should ideally belong to the same category.
    """
    # Check SKU-based classification first
    sku = str(row['sku']).upper() if pd.notna(row['sku']) else ''
    if sku.startswith('TPL'):
        return 'Network'
    
    if sku.startswith('PLA'):
        return 'Headsets'

    #if sku.startswith('IKM'):
    #    return 'Audio Equipment'


    type = str(row['type']) if pd.notna(row['type']) else ''
    #types I regognized
    if type=="11935397":
        return 'Memory & Storage'
    
    if type=="11865403":
        return 'Cases & Protection'
    
    #if type=="11905404":
    #    return 'Audio Equipment'
    
    # Clean name by removing condition-related prefixes
    name_value = str(row['name'])
    name_clean = re.sub(r'^(Open -|Like new -)\s*', '', name_value, flags=re.IGNORECASE)


        # Clean description by removing refurbishment keywords
    desc_value = row.get('desc', '')
    if pd.isna(desc_value):
        desc_value = ''
    desc_clean = re.sub(r'\b(Refurbished|Reconditioned)\b', '', str(desc_value), flags=re.IGNORECASE)
    
    # Combine cleaned name and description for analysis (convert to lowercase)
    text = name_clean.lower() + ' ' + desc_clean.lower()
    price = float(row['price']) if pd.notna(row['price']) else 0
    
    # Normalize description for prefix-based rules
    desc_text = desc_clean.strip().lower()

    if 'switch' in text and 'port' in text:
        return 'Network'

    
    # Define category keywords and patterns
    # Wearables: Apple Watch devices and bundles marked by leading description
    if desc_text.startswith('apple watch') or re.match(r'^\d{1,3}mm apple watch', desc_text):
        return 'Wearables'
    
    # Wearables: Bluetooth smart watches
    if 'bluetooth smart watch' in text or 'bluetooth smartwatch' in text:
        return 'Wearables'
    

    # Stands and mounts for phones/tablets
    if any(word in text for word in ['stand', 'support', 'mount', 'dock', 'docking station', 'holder',
                                        'base', 'tripod']):
        return 'Stands & Mounts'
    
    # AirPods: Apple wireless earbuds
    if any(word in text for word in ['headphone', 'headset', 'earphone', 'airpods', 'airpod']):
        return 'Headsets'
    
    # Memory devices called out by size-first SSD descriptions
    if (re.match(r'^\d+\s*gb ssd', desc_text) or 
        re.match(r'^\d+(?:\.\d+)?\s*tb ssd', desc_text) or 
        desc_text.startswith('ssd hard drive')):
        return 'Memory & Storage'
    
    # Services (explicit service products only; ignore generic warranty mentions like "xx years warranty")
    if any(word in text for word in ['applecare', 'apple care', 'protection plan', 
                                      'service plan', 'extended warranty', 'care pack', 'repair service']):
        return 'Services'
    
    # Software and licenses
    if any(word in text for word in ['license', 'software', 'pack license']):
        return 'Software & Licenses'
    
    # Extension/Expansion kits are memory (check before generic 'case' rule)
    if any(word in text for word in ['extension kit', 'expansion kit']):
        return 'Memory & Storage'
    
    # Cases and accessories first (more specific, to avoid misclassification)
    if any(word in text for word in ['case', 'cover', 'sleeve', 'housing', 'shell', 'protector']):
        return 'Cases & Protection'
    
    # Battery cases usually have higher price but still cases
    if any(word in text for word in ['battery case', 'juice pack']):
        return 'Cases & Protection'
    
    # Dedicated memory kits (place before NAS/servers to avoid misclassifying RAM upgrades)
    if any(word in text for word in ['memory for', 'ram for', 'memory upgrade for']):
        return 'Memory & Storage'

    # Server / NAS devices
    if any(word in text for word in ['nas', 'rackstation', 'synology', 'qnap', 'server']):
        return 'Servers & NAS'

    # Computers (laptops/desktops/all-in-one) before memory so they don't fall into RAM keywords
    if any(word in text for word in ['macbook pro', 'macbook air', 'macbook', 'imac', 'mac mini', 'macmini', 'mac pro', 'laptop', 'notebook', 'desktop', 'tower']) and price > 400:
        return 'Computers'

    # Now check for actual phones (should be expensive, >200)
    if any(word in text for word in ['iphone', 'phone', 'smartphone']) and price > 300:
        return 'Phones'
    
    # Tablets (should be expensive, >150)
    if any(word in text for word in ['ipad', 'tablet']) and price > 250:
        return 'Tablets'
    
    if any(word in text for word in ['helicopter', 'toy',  'drone']):
        return 'Toys & Entertainment'

    
    if any(word in text for word in ['keyboard', 'keypad', 'key pad']):
        return 'Keyboards & Input Devices'
    
    if any(word in text for word in ['mouse', 'trackpad', 'magic trackpad']):
        return 'Keyboards & Input Devices'
    
    if any(word in text for word in ['jetdrive', 'jumpdrive', 'memory for']):
        return 'Memory & Storage'

    if any(word in text for word in ['ram', 'dimm', 'sodimm', 'so-dimm', 'ddr', 'ecc', 'pc3', 'pc2', 'pc4', 'memory kit', 'memory module']):
        return 'Memory & Storage'

    if any(word in text for word in ['adapter', 'connector']):
        return 'Cables & Adapters'

    if any(word in text for word in ['pendrive', 'flash drive' 'hard drive', 'hdd', 'ssd', 'storage', 'barracuda', 'external hard', 'harddisk', 'nvme', 'm.2', 'solid state']):
        return 'Memory & Storage'
    
    if any(word in text for word in ['cable', 'adapter', 'connector', 'usb', 'hdmi', 'vga', 'dvi', 
                                        'thunderbolt', 'firewire', 'displayport', 'mini display',
                                        'ethernet adapter', 'dock connector']):
        return 'Cables & Adapters'
    
    if any(word in text for word in ['charger', 'magsafe', 'power', 'battery', 'charging']):
        return 'Power & Charging'
    
    if any(word in text for word in [ 'audio', 'speaker', 'microphone',
                                        'music receiver', 'mic']):
        return 'Audio Equipment'
    
    if any(word in text for word in ['monitor', 'display', 'screen']) and price > 100:
        return 'Monitors & Displays'
    
    if any(word in text for word in ['stylus', 'pen', 'wacom', 'bamboo', 'graphics tablet', 'intuos']):
        return 'Graphics & Drawing Tools'
    
    if any(word in text for word in ['lock', 'security', 'theft', 'clicksafe', 'antitheft']):
        return 'Security & Locks'
    
    if any(word in text for word in ['camera', 'lens', 'photo', 'video']):
        return 'Camera & Photo'
    
    if any(word in text for word in ['cleaner', 'cleaning', 'maintenance']):
        return 'Cleaning & Maintenance'
    
    if any(word in text for word in ['scale', 'wireless scale']):
        return 'Health & Fitness'
    
    if any(word in text for word in ['guitar', 'dj', 'mixer', 'irig', 'music production']):
        return 'Audio Equipment'
    
    
    if any(word in text for word in ['apple tv', 'media player']):
        return 'Media Players'
    
    # Catch remaining phone/tablet accessories (cheap items with phone/tablet keywords)
    if any(word in text for word in ['iphone', 'ipad', 'ipod', 'phone', 'tablet']):
        return 'Phone & Tablet Accessories'
    

    return 'Accessories'


def check_if_refurbished(row):
    """
    Check if a product is refurbished based on name and description.
    Returns 'Refurbished' or 'New'.
    """
    text = str(row['name']).lower() + ' ' + str(row['desc']).lower()
    
    # Keywords indicating refurbished items
    refurbished_keywords = [
        'refurbished', 'refurb', 'renewed', 'reconditioned', 
        'used', 'open box', 'oem', 'unboxed', 'pre-owned',
        'second hand', 'secondhand', 'reacondicionado', 'open -', 'like new -'

    ]
    
    if any(keyword in text for keyword in refurbished_keywords):
        return 'Refurbished'
    else:
        return 'New'


# Apply the categorization functions
products_cl['category'] = products_cl.apply(categorize_product, axis=1)
products_cl['condition'] = products_cl.apply(check_if_refurbished, axis=1)

# Display the results
print("Category distribution:")
print(products_cl['category'].value_counts())
print("\n" + "="*80 + "\n")

print("Condition distribution:")
print(products_cl['condition'].value_counts())
print("\n" + "="*80 + "\n")

# Show sample products from each category with prices
print("Sample products by category:")
for category in sorted(products_cl['category'].unique()):
    print(f"\n{category}:")
    sample = products_cl[products_cl['category'] == category][['sku', 'name', 'price', 'condition', 'type']].head(3)
    for idx, row in sample.iterrows():
        print(f"  - {row['sku']}: {row['name']} (${row['price']}, {row['condition']}, type: {row['type']})")

# Show price statistics by category
print("\n" + "="*80 + "\n")
print("Price statistics by category:")
price_stats = products_cl.groupby('category')['price'].agg(['count', 'mean', 'min', 'max']).round(2)
print(price_stats.sort_values('mean', ascending=False))

# Show refurbished items
print("\n" + "="*80 + "\n")
print("Sample refurbished items:")
refurbished = products_cl[products_cl['condition'] == 'Refurbished'][['sku', 'name', 'price', 'category']].head(10)
for idx, row in refurbished.iterrows():
    print(f"  - {row['sku']}: {row['name']} (${row['price']}, {row['category']})")

# Check if products with same 'type' have the same category
print("\n" + "="*80 + "\n")
print("Checking consistency: Products with same 'type' value:")
type_category_map = products_cl.groupby('type')['category'].unique()
inconsistent_types = type_category_map[type_category_map.apply(len) > 1]

if len(inconsistent_types) > 0:
    print(f"\nFound {len(inconsistent_types)} type values with multiple categories:")
    for type_val, categories in inconsistent_types.head(10).items():
        print(f"\nType {type_val}:")
        for cat in categories:
            count = len(products_cl[(products_cl['type'] == type_val) & (products_cl['category'] == cat)])
            print(f"  - {cat}: {count} products")
else:    print("All products with the same 'type' value belong to the same category!")

# Show breakdown by category and conditionprint(pd.crosstab(products_cl['category'], products_cl['condition']))

print("\n" + "="*80 + "\n")
print("Products by category and condition:")

IndentationError: expected an indented block after 'if' statement on line 119 (624262971.py, line 120)